# DIorDIE analysis
The experiment: **Compare different dependency injection solutions available for Kotlin and measure startup performance**

* [spring](../di-test-spring)
* [koin](../di-test-koin)
* [manual](../di-test-manual)
*
To test on your own, first run `gradlew runRepeated -Pcycles=<desired cycles>` on each project, the outputs are then in `<project>/build/outputs/repeated-runs.txt`

## Analysis
Let's copy all the outputs  to the `data` folder and consolidate into CSV.

In [2]:
import java.io.File
import java.nio.file.Path
import java.nio.file.Files
import java.nio.file.StandardCopyOption

val diTypes = listOf("spring", "koin", "manual")
val dataMap = mutableMapOf<String, MutableList<String>>()

// Copy files
diTypes.forEach { diType ->
    val copied = Path.of("$diType.txt")
    Files.copy(
        Path.of("../di-test-$diType/build/outputs/repeated-runs.txt"),
        Path.of("$diType.txt"),
        StandardCopyOption.REPLACE_EXISTING
    )

    copied.toFile().readLines().forEachIndexed { index, line ->
        dataMap.computeIfAbsent(index.toString()) { mutableListOf() }.add(line)
    }
}

// Consolidate files into one CSV
File("consolidated.csv").printWriter().use { writer ->
    writer.println(diTypes.joinToString(","))
    dataMap.values.forEach { row ->
        writer.println(row.joinToString(","))
    }
}




Analyze the results, compute aggregated metrics (avg, med, some percentils) and compare.

In [1]:
%useLatestDescriptors
%use dataframe
%use kandy
import org.jetbrains.kotlinx.dataframe.api.*
import org.jetbrains.kotlinx.statistics.dataframe.stat.mean
import org.jetbrains.letsPlot.Geom
import org.jetbrains.letsPlot.core.spec.plotson.dodge
import org.jetbrains.letsPlot.core.spec.plotson.plot

// Read consolidated CSV into a DataFrame
val df = DataFrame.read(File("consolidated.csv"))

// Combine mean and median directly into a single DataFrame
val combinedSummaryDf = dataFrameOf(
    "Metric" to listOf("Mean", "Median", "Perc80"),
    df.columnNames().get(0) to listOf(
        df.columns().get(0).convertTo<Double>().mean(),
        df.columns().get(0).convertTo<Double>().median(),
        df.columns().get(0).convertTo<Double>().percentile(80.0)
    ),
    df.columnNames().get(1) to listOf(
        df.columns().get(1).convertTo<Double>().mean(),
        df.columns().get(1).convertTo<Double>().median(),
        df.columns().get(1).convertTo<Double>().percentile(80.0)
    ),
    df.columnNames().get(2) to listOf(
        df.columns().get(2).convertTo<Double>().mean(),
        df.columns().get(2).convertTo<Double>().median(),
        df.columns().get(2).convertTo<Double>().percentile(80.0)
    )
)

// Print the combined DataFrame
combinedSummaryDf.print()
val meltedDf = combinedSummaryDf.gather {
    cols("spring", "koin", "manual")
}.into("Implementation", "Value").update("Value") { it as Double / 1000000 }

plot(meltedDf) {
    bars {
        x("Metric")
        y("Value")
        fillColor("Implementation")
        position = Position.dodge()
    }
    text {
        x("Metric")
        y("Value")
        label("Value")
        position = Position.dodge()
    }
//    title("Comparison of Metrics: Spring vs Koin vs Manual")
}

   Metric           spring            koin       manual
 0   Mean 422102403,048000 25531831,958000 20127158,926
 1 Median 409085797,000000 22788585,000000 18869805,000
 2 Perc80 452796269,400000 29335791,200000 22447169,800



<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="T5FoZo"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
"Value":[422.10240304800004,25.531831958000048,20.127158926,409.085797,22.788585,18.869805,452.7962694,29.335791200000006,22.4471698],
"Metric":["Mean","Mean","Mean","Median","Median","Median","Perc80","Perc80","Perc80"],
"Implementation":["spring","koin","manual","spring","koin","manual","spring","koin","manual"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"Metric",
"y":"Value",
"fill":"Implementation"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
},{
"mapping":{
"x":"Metric",
"y":"Value",
"label":"Value"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"text",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"Metric"
},{
"type":"float",
"column":"Value"
},{
"type":"str",
"column":"Implementation"
}]
},
"spec_id":"2"
};
 var containerDiv = document.getElementById("T5FoZo");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 422.102 
 
 
 
 
 
 
 25.5318 
 
 
 
 
 
 
 20.1272 
 
 
 
 
 
 
 409.086 
 
 
 
 
 
 
 22.7886 
 
 
 
 
 
 
 18.8698 
 
 
 
 
 
 
 452.796 
 
 
 
 
 
 
 29.3358 
 
 
 
 
 
 
 22.4472 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Mean 
 
 
 
 
 
 
 
 
 Median 
 
 
 
 
 
 
 
 
 Perc80 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 150 
 
 
 
 
 
 
 200 
 
 
 
 
 
 
 250 
 
 
 
 
 
 
 300 
 
 
 
 
 
 
 350 
 
 
 
 
 
 
 400 
 
 
 
 
 
 
 450 
 
 
 
 
 
 
 
 
 Value 
 
 
 
 
 Metric 
 
 
 
 
 
 
 
 
 Implementation 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 spring 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 koin 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 manual

In [ ]:
%useLatestDescriptors
%use kandy